📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/pytorch-workshop](https://github.com/mr-pylin/pytorch-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Linear Regression](#toc2_)    
  - [Load Boston Housing Dataset](#toc2_1_)    
  - [Implementation 1](#toc2_2_)    
  - [Implementation 2](#toc2_3_)    
  - [Implementation 3](#toc2_4_)    
  - [Implementation 4](#toc2_5_)    
- [Logistic Regression](#toc3_)    
  - [Load Breast Cancer Dataset](#toc3_1_)    
  - [Implementation](#toc3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn, optim

In [2]:
# set a seed for deterministic results
seed = 42

# <a id='toc2_'></a>[Linear Regression](#toc0_)

- **Linear Regression** is a **supervised** machine learning algorithm.
- It's used to model the **relationship** between a dependent variable (**target**) and one or more independent variables (**features**).
- It predicts a **continuous** output.

<figure style="text-align:center; margin:0;">
  <img src="../assets/images/original/perceptron/linear-regression.svg" alt="linear-regression.svg" style="max-width:80%; height:auto;">
  <figcaption style="text-align:center;">Linear Regression Model</figcaption>
</figure>


## <a id='toc2_1_'></a>[Load Boston Housing Dataset](#toc0_)


In [ ]:
# boston dataset as a pandas data-frame
boston_df = pd.read_csv(
    "https://raw.githubusercontent.com/mr-pylin/datasets/refs/heads/main/data/tabular-data/boston-housing/dataset.csv",
    encoding="utf-8",
)

# log
print(boston_df.head())

In [ ]:
# separate features and target
X = torch.tensor(boston_df.drop(columns=["MEDV"]).values, dtype=torch.float32)
y = torch.tensor(boston_df["MEDV"].values, dtype=torch.float32)

# split dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

# standardize features
x_train_mean = x_train.mean(dim=0)
x_train_std = x_train.std(dim=0)
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# reshape targets to add `batch` dimension
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# log
print(f"x_train[0]: {x_train[0]}\n")
print(f"y_train[0]: {y_train[0]}")

## <a id='toc2_2_'></a>[Implementation 1](#toc0_)

<table style="text-align: center; border-collapse: collapse;">
  <tr>
    <th style="width: 25%;">Feedforward</th>
    <th style="width: 25%;">Gradient Computation</th>
    <th style="width: 25%;">Loss Computation</th>
    <th style="width: 25%;">Parameter Update</th>
  </tr>
  <tr>
    <td>Manual ❌</td>
    <td>Manual ❌</td>
    <td>Manual ❌</td>
    <td>Manual ❌</td>
  </tr>
</table>


In [ ]:
# initialize weights and bias
torch.manual_seed(seed)
w = torch.randn((1, x_train.shape[1]), dtype=torch.float32)
b = torch.zeros((1), dtype=torch.float32)

# hyperparameters
lr = 0.02
epochs = 100


# feed-forward
def model_1(x: torch.Tensor) -> torch.Tensor:
    return x @ w.T + b


# MSE loss
def loss(y_pred: torch.Tensor, y_train: torch.Tensor) -> torch.Tensor:
    return ((y_pred - y_train) ** 2).mean()


# gradients for weights and bias
def gradient(x: torch.Tensor, y_pred: torch.Tensor, y_train: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    dw = (2 * ((y_pred - y_train).T @ x)) / x.shape[0]
    db = (2 * (y_pred - y_train)).mean()
    return dw, db


# training loop
for epoch in range(epochs):

    # feed-forward
    y_pred = model_1(x_train)

    # loss
    l = loss(y_pred, y_train)

    # backward
    dw, db = gradient(x_train, y_pred, y_train)

    # update parameters
    w -= lr * dw
    b -= lr * db

    # log
    if epoch % 10 == 0 or (epoch + 1) == epochs:
        print(f"epoch {epoch+1:0{len(str(epochs))}}/{epochs} -> loss: {l:9.5f}")

In [ ]:
# predict test set targets
with torch.no_grad():
    y_pred = model_1(x_test)

for i in range(len(y_test)):
    print(f"[y_true: {y_test[i].squeeze():8.5f} | y_pred: {y_pred[i].squeeze():8.5f}]", end="")
    if (i + 1) % 4 == 0:
        print("")
    else:
        print(" , ", end="")

## <a id='toc2_3_'></a>[Implementation 2](#toc0_)

<table style="text-align: center; border-collapse: collapse;">
  <tr>
    <th style="width: 25%;">Feedforward</th>
    <th style="width: 25%;">Gradient Computation</th>
    <th style="width: 25%;">Loss Computation</th>
    <th style="width: 25%;">Parameter Update</th>
  </tr>
  <tr>
    <td>Manual ❌</td>
    <td>Auto ✅</td>
    <td>Manual ❌</td>
    <td>Manual ❌</td>
  </tr>
</table>


In [ ]:
# initialize weights and bias
torch.manual_seed(seed)
w = torch.randn((1, x_train.shape[1]), dtype=torch.float32, requires_grad=True)
b = torch.zeros((1), dtype=torch.float32, requires_grad=True)

# hyperparameters
lr = 0.02
epochs = 100


# feed-forward
def model_2(x: torch.Tensor) -> torch.Tensor:
    return x @ w.T + b


# MSE loss
def loss(y_pred: torch.Tensor, y_train: torch.Tensor) -> torch.Tensor:
    return ((y_pred - y_train) ** 2).mean()


# training loop
for epoch in range(epochs):

    # feed-forward
    y_pred = model_2(x_train)

    # loss
    l = loss(y_pred, y_train)

    # backward
    l.backward()

    # update parameters
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad

        # zero the gradients to prevent accumulating
        w.grad.zero_()
        b.grad.zero_()

    # log
    if epoch % 10 == 0 or (epoch + 1) == epochs:
        print(f"epoch {epoch+1:0{len(str(epochs))}}/{epochs} -> loss: {l:9.5f}")

In [ ]:
# predict test set targets
with torch.no_grad():
    y_pred = model_2(x_test)

for i in range(len(y_test)):
    print(f"[y_true: {y_test[i].squeeze():8.5f} | y_pred: {y_pred[i].squeeze():8.5f}]", end="")
    if (i + 1) % 4 == 0:
        print("")
    else:
        print(" , ", end="")

## <a id='toc2_4_'></a>[Implementation 3](#toc0_)

<table style="text-align: center; border-collapse: collapse;">
  <tr>
    <th style="width: 25%;">Feedforward</th>
    <th style="width: 25%;">Gradient Computation</th>
    <th style="width: 25%;">Loss Computation</th>
    <th style="width: 25%;">Parameter Update</th>
  </tr>
  <tr>
    <td>Manual ❌</td>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
  </tr>
</table>


In [ ]:
# initialize weights and bias
torch.manual_seed(seed)
w = torch.randn((1, x_train.shape[1]), dtype=torch.float32, requires_grad=True)
b = torch.zeros((1), dtype=torch.float32, requires_grad=True)

# hyperparameters
lr = 0.02
epochs = 100

# initialize criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD([w, b], lr=lr)


# feed-forward
def model_3(x: torch.Tensor) -> torch.Tensor:
    return x @ w.T + b


# training loop
for epoch in range(epochs):

    # feed-forward
    y_pred = model_3(x_train)

    # loss
    l = criterion(y_pred, y_train)

    # backward
    l.backward()

    # update parameters
    optimizer.step()
    optimizer.zero_grad()

    # log
    if epoch % 10 == 0 or (epoch + 1) == epochs:
        print(f"epoch {epoch+1:0{len(str(epochs))}}/{epochs} -> loss: {l:9.5f}")

In [ ]:
# predict test set targets
with torch.no_grad():
    y_pred = model_3(x_test)

for i in range(len(y_test)):
    print(f"[y_true: {y_test[i].squeeze():8.5f} | y_pred: {y_pred[i].squeeze():8.5f}]", end="")
    if (i + 1) % 4 == 0:
        print("")
    else:
        print(" , ", end="")

## <a id='toc2_5_'></a>[Implementation 4](#toc0_)

<table style="text-align: center; border-collapse: collapse;">
  <tr>
    <th style="width: 25%;">Feedforward</th>
    <th style="width: 25%;">Gradient Computation</th>
    <th style="width: 25%;">Loss Computation</th>
    <th style="width: 25%;">Parameter Update</th>
  </tr>
  <tr>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
  </tr>
</table>


In [ ]:
# define a linear transformation
model_4 = nn.Linear(in_features=x_train.shape[1], out_features=1)

# initialize weights and bias [to start from the point of above implementations]
torch.manual_seed(seed)
with torch.no_grad():
    model_4.weight = nn.init.normal_(model_4.weight, mean=0, std=1)
    model_4.bias = nn.init.zeros_(model_4.bias)

# hyperparameters
lr = 0.02
epochs = 100

# initialize criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model_4.parameters(), lr=lr)

# training loop
for epoch in range(epochs):

    # feed-forward
    y_pred = model_4(x_train)

    # loss
    l = criterion(y_pred, y_train)

    # backward
    l.backward()

    # update parameters
    optimizer.step()
    optimizer.zero_grad()

    # log
    if epoch % 10 == 0 or (epoch + 1) == epochs:
        print(f"epoch {epoch+1:0{len(str(epochs))}}/{epochs} -> loss: {l:9.5f}")

In [ ]:
# predict test set targets
with torch.no_grad():
    y_pred = model_4(x_test)

for i in range(len(y_test)):
    print(f"[y_true: {y_test[i].squeeze():8.5f} | y_pred: {y_pred[i].squeeze():8.5f}]", end="")
    if (i + 1) % 4 == 0:
        print("")
    else:
        print(" , ", end="")

# <a id='toc3_'></a>[Logistic Regression](#toc0_)

- **Logistic Regression** is a **supervised** machine learning algorithm.
- It models the **relationship** between a dependent variable (**target**) and one or more independent variables (**features**).
- Unlike **linear regression**, it predicts a **categorical outcome**, typically for **binary classification** (0 or 1).
- It uses the **logistic** function (**sigmoid**) to map predicted values to **probabilities**.

<figure style="text-align:center; margin:0;">
  <img src="../assets/images/original/perceptron/logistic-regression.svg" alt="logistic-regression.svg" style="max-width:80%; height:auto;">
  <figcaption style="text-align:center;">Logistic Regression Model</figcaption>
</figure>


## <a id='toc3_1_'></a>[Load Breast Cancer Dataset](#toc0_)


In [ ]:
# breast cancer as a pandas data-frame
breast_df = pd.read_csv(
    "https://raw.githubusercontent.com/mr-pylin/datasets/refs/heads/main/data/tabular-data/breast-cancer-wisconsin-diagnostic/dataset.csv",
    encoding="utf-8",
)

# encode labels to numbers
breast_df["Diagnosis"] = breast_df["Diagnosis"].map({"B": 0, "M": 1})

# log
print(breast_df.head())

In [ ]:
# separate features and target
X = torch.tensor(breast_df.iloc[:, 2:].values, dtype=torch.float32)
y = torch.tensor(breast_df["Diagnosis"].values, dtype=torch.float32)

# split dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

# standardize features
x_train_mean = x_train.mean(dim=0)
x_train_std = x_train.std(dim=0)
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# reshape targets to add `batch` dimension
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# log
print(f"x_train[0]: {x_train[0]}\n")
print(f"y_train[0]: {y_train[0]}")

## <a id='toc3_2_'></a>[Implementation](#toc0_)

<table style="text-align: center; border-collapse: collapse;">
  <tr>
    <th style="width: 25%;">Feedforward</th>
    <th style="width: 25%;">Gradient Computation</th>
    <th style="width: 25%;">Loss Computation</th>
    <th style="width: 25%;">Parameter Update</th>
  </tr>
  <tr>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
    <td>Auto ✅</td>
  </tr>
</table>


In [ ]:
# define a linear transformation
model = nn.Sequential(
    nn.Linear(in_features=x_train.shape[1], out_features=1),
    nn.Sigmoid(),
)

# log
model

In [ ]:
# hyperparameters
lr = 0.02
epochs = 100

# initialize criterion and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# training loop
for epoch in range(epochs):

    # feed-forward
    y_pred = model(x_train)

    # loss
    l = criterion(y_pred, y_train)

    # backward
    l.backward()

    # update parameters
    optimizer.step()
    optimizer.zero_grad()

    # compute accuracy
    acc = ((y_pred >= 0.5).float() == y_train).float().mean().item()

    # log
    if epoch % 10 == 0 or (epoch + 1) == epochs:
        print(f"epoch {epoch+1:0{len(str(epochs))}}/{epochs} -> loss: {l:7.5f} | acc: {acc*100:5.2f}%")

In [ ]:
# test
with torch.no_grad():

    # feed-forward
    y_pred = model(x_test)

    # loss
    l = criterion(y_pred, y_test)

    # compute accuracy
    acc = ((y_pred >= 0.5).float() == y_test).float().mean().item()

    # log
    if epoch % 10 == 0 or (epoch + 1) == epochs:
        print(f"loss: {l:9.5f} | acc: {acc*100:5.2f}%")